In [1]:
import mindquantum

print(mindquantum.__version__)

0.9.0


# CHSH

$$
\begin{align}
S_{CHSH} & = X \otimes \left(Y + Y' \right) + X' \otimes (Y - Y') \\
& = \vec{a} \cdot \vec{\sigma} \otimes \left(\vec{b}\cdot \vec{\sigma} + \vec{b'} \cdot \vec{\sigma} \right) 
+ \vec{a'} \cdot \vec{\sigma} \otimes (\vec{b} \cdot \vec{\sigma} - \vec{b'} \cdot \vec{\sigma})
\end{align}
$$

where $\vec{a},\vec{b}, \vec{a'}, \vec{b'} \in \mathbb{R}^3$ and $|\vec{a}| = |\vec{b}| = |\vec{a'}| = |\vec{b'}| = 1$ .

In [2]:
from mindspore import nn
from mindspore import ops
from mindspore import Tensor, Parameter

import mindspore as ms
ms.set_context(mode=ms.PYNATIVE_MODE)

import numpy as np

In [3]:
class ClassicalNet2(nn.Cell):
    def __init__(self):
        super().__init__()

        def build(prefix: str):
            minval = Tensor(0, ms.float32)
            maxval = Tensor(np.pi * 2, ms.float32)
            theta = Parameter(ops.uniform((1,), minval, maxval, dtype=ms.float32), name=prefix+"_theta", requires_grad=True)
            phi   = Parameter(ops.uniform((1,), minval, maxval, dtype=ms.float32), name=prefix+"_phi"  , requires_grad=True)
            self.__setattr__(f"{prefix}_theta", theta)
            self.__setattr__(f"{prefix}_phi", phi)

        build("a^1")
        build("a^2")
        build("b^1")
        build("b^2")
        self.test = [Parameter(np.random.rand(2,), name="test", requires_grad=True)]
   
    def construct(self, T: Tensor):
        def get_vec(prefix: str):
            theta = self.__getattr__(f"{prefix}_theta")
            phi   = self.__getattr__(f"{prefix}_phi")
            vec = ops.concat((ops.cos(theta) * ops.sin(phi), ops.sin(theta) * ops.sin(phi), ops.cos(phi)), axis=0)
            return vec

        def f(a: Tensor, b: Tensor):
            middle = T * a
            output = middle.transpose() * b
            return output.transpose()
        
        a  = get_vec("a^1")
        a_ = get_vec("a^2")
        b  = get_vec("b^1")
        b_ = get_vec("b^2")
        return ops.concat((f(a, b), f(a, b_), f(a_, b), -f(a_, b_)), axis=0).sum()


In [4]:
class QuantumNet2:
    def __call__(self, rho: np.ndarray):
        from mindquantum.simulator import Simulator
        from mindquantum.core.operators import Hamiltonian, QubitOperator
        sim = Simulator("mqmatrix", 2)
        sim.set_qs(rho)
        T = np.zeros(shape=(3,3))
        for (i, sigma1) in enumerate(["X", "Y", "Z"]):
            for (j, sigma2) in enumerate(["X", "Y", "Z"]):
                ham = Hamiltonian(QubitOperator(f"{sigma1}0 {sigma2}1"))
                T[i, j] = sim.get_expectation(ham).real
        return Tensor(T, dtype=ms.float32)


In [5]:
def train2(rho: np.matrix, EPOCHS=1000, lr=1e-2):
    q_nn = QuantumNet2()
    rho = q_nn(np.asarray(rho))

    model = ClassicalNet2()
    optimizer = nn.Adam(model.trainable_params(), learning_rate=lr)
    # optimizer = nn.SGD(model.trainable_params(), learning_rate=lr)

    def loss_fn(x, y):
        return -x

    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss
    
    model.set_train()
    for epoch in range(EPOCHS):
        loss = train_step(rho, None)
        if epoch % 100 == 0:
            loss, current = loss.asnumpy(), epoch
            print(f"loss: {loss:>7f}  [{current:>3d} / {EPOCHS:>3d}]")

            (loss, _), grads = grad_fn(rho, None)
            print(f"loss: {loss} grads: {grads}")
    return train_step(rho, None).asnumpy()

In [6]:
rho = np.array([[0, 0, 0, 0],
                [0, 1, -1, 0],
                [0, -1, 1, 0],
                [0, 0, 0, 0]])
train2(rho, 1000, 0.01)

loss: 0.325064  [  0 / 1000]
loss: 0.27400205 grads: (Tensor(shape=[1], dtype=Float32, value= [-1.93436086e-01]), Tensor(shape=[1], dtype=Float32, value= [-1.11425364e+00]), Tensor(shape=[1], dtype=Float32, value= [ 8.41760039e-02]), Tensor(shape=[1], dtype=Float32, value= [-1.05288541e+00]), Tensor(shape=[1], dtype=Float32, value= [ 4.05643284e-01]), Tensor(shape=[1], dtype=Float32, value= [ 4.51240800e-02]), Tensor(shape=[1], dtype=Float32, value= [-2.96383202e-01]), Tensor(shape=[1], dtype=Float32, value= [ 1.91997087e+00]))
loss: -2.714450  [100 / 1000]
loss: -2.7195542 grads: (Tensor(shape=[1], dtype=Float32, value= [ 1.25910789e-02]), Tensor(shape=[1], dtype=Float32, value= [-4.90504324e-01]), Tensor(shape=[1], dtype=Float32, value= [ 1.24595985e-02]), Tensor(shape=[1], dtype=Float32, value= [-8.90788063e-02]), Tensor(shape=[1], dtype=Float32, value= [-4.84205782e-02]), Tensor(shape=[1], dtype=Float32, value= [-1.33770704e-02]), Tensor(shape=[1], dtype=Float32, value= [ 2.3369908

array(-2.828427, dtype=float32)

In [6]:
rho = np.array([[0, 0, 0, 0],
                [0, 1, -1, 0],
                [0, -1, 1, 0],
                [0, 0, 0, 0]])
train2(rho, 1000, 0.01)


[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
loss: -1.821874  [  0 / 1000]
loss: -1.8403138 grads: (Tensor(shape=[1], dtype=Float32, value= [-3.78581658e-02]), Tensor(shape=[1], dtype=Float32, value= [-8.03005159e-01]), Tensor(shape=[1], dtype=Float32, value= [ 4.39323485e-02]), Tensor(shape=[1], dtype=Float32, value= [ 9.54542756e-01]), Tensor(shape=[1], dtype=Float32, value= [ 1.50001198e-01]), Tensor(shape=[1], dtype=Float32, value= [-4.44560885e-01]), Tensor(shape=[1], dtype=Float32, value= [-1.56075373e-01]), Tensor(shape=[1], dtype=Float32, value= [-4.93906103e-02]))
loss: -2.670384  [100 / 1000]
loss: -2.6738584 grads: (Tensor(shape=[1], dtype=Float32, value= [-3.91222239e-02]), Tensor(shape=[1], dtype=Float32, value= [-4.41737443e-01]), Tensor(shape=[1], dtype=Float32, value= [ 5.55173978e-02]), Tensor(shape=[1], dtype=Float32, value= [ 2.29631126e-01]), Tensor(shape=[1], dtype=Float32, value= [-6.19477034e-03]), Tensor(shape=[1], dtype=Float32, value= [-7.18493462e-02]), Tens

Tensor(shape=[], dtype=Float32, value= -2.82843)

In [9]:
def rand_state(n_qubits: int) -> np.ndarray:
    d = 2**n_qubits
    re = np.random.random((d,))
    im = np.random.random((d,))
    state = re + 1.0j * im
    return state / np.linalg.norm(state)

def rand_density_matrix(n_qubits: int, m: int) -> np.matrix:
    d = 2**n_qubits
    probabilities = np.random.random((m, ))
    probabilities /= probabilities.sum()

    rho = np.zeros((d, d), dtype=np.complex128)
    rho = np.asmatrix(rho)
    for p in probabilities:
        state = np.asmatrix(rand_state(n_qubits))
        rho += p * state.H @ state
    return rho

In [25]:
v = rand_density_matrix(2, 3)
assert(np.allclose(v.H, v))
assert(np.allclose(1.0, v.trace()[0,0]))

In [11]:
def CHSH_max(rho: np.matrix):
    X = np.matrix([[0, 1], [1, 0]], dtype=np.complex128)
    Y = np.matrix([[0, -1j], [1j, 0]], dtype=np.complex128)
    Z = np.matrix([[1, 0], [0, -1]], dtype=np.complex128)
    sigma = [X, Y, Z]
    T = [[(rho @ np.kron(sigma[i], sigma[j])).trace()[0, 0] for j in range(3)] for i in range(3)]
    T = np.matrix(T).real
    U = T.T @ T
    eigens = np.linalg.eigvals(U)
    eigens.sort()
    return 2.0 * np.sqrt(eigens[-1] + eigens[-2])

In [12]:
CHSH_max(rho)

2.8284271247461903

In [29]:
for i in range(1):
    rho = rand_density_matrix(2, 1)
    v1 = CHSH_max(rho)
    print(f"CHSH max: {v1:.7f}")
    v2 = train2(rho)[0]
    print(f"estimate: {v2:.7f}")
    if not np.allclose(v1, v2):
        print(f"Wrong!\nrho:{rho}")
        break

CHSH max: 2.0587074
loss: -1.638395  [  0 / 1000]
loss: -1.701862  [ 10 / 1000]
loss: -1.737221  [ 20 / 1000]
loss: -1.745391  [ 30 / 1000]
loss: -1.728553  [ 40 / 1000]
loss: -1.689875  [ 50 / 1000]
loss: -1.633124  [ 60 / 1000]
loss: -1.562286  [ 70 / 1000]
loss: -1.481230  [ 80 / 1000]
loss: -1.393472  [ 90 / 1000]
loss: -1.302033  [100 / 1000]
loss: -1.209388  [110 / 1000]
loss: -1.117480  [120 / 1000]
loss: -1.027778  [130 / 1000]
loss: -0.941353  [140 / 1000]
loss: -0.858970  [150 / 1000]
loss: -0.781160  [160 / 1000]
loss: -0.708301  [170 / 1000]
loss: -0.640666  [180 / 1000]
loss: -0.578473  [190 / 1000]
loss: -0.521909  [200 / 1000]
loss: -0.471151  [210 / 1000]
loss: -0.426368  [220 / 1000]
loss: -0.387718  [230 / 1000]
loss: -0.355330  [240 / 1000]
loss: -0.329290  [250 / 1000]
loss: -0.309612  [260 / 1000]
loss: -0.296210  [270 / 1000]
loss: -0.288877  [280 / 1000]
loss: -0.287257  [290 / 1000]
loss: -0.290833  [300 / 1000]
loss: -0.298925  [310 / 1000]
loss: -0.310698  [32

ValueError: Expect Tensor to have dimension between 1 and 8.

In [52]:
import mindspore

print(mindspore.__version__)

1.10.1


In [16]:
a = Tensor((1, 2, 3))
a

Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])

In [17]:
t = Tensor(np.array([[1, 2, 3], [4,5,6], [7,8,9]]))
t

Tensor(shape=[3, 3], dtype=Int64, value=
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]])

In [18]:
t[2, :] * a

Tensor(shape=[3], dtype=Int64, value= [ 7, 16, 27])

In [20]:
ops.stack((t[0, :] * a, t[1, :] * a, t[2, :] * a))

Tensor(shape=[3, 3], dtype=Int64, value=
[[ 1,  4,  9],
 [ 4, 10, 18],
 [ 7, 16, 27]])

In [23]:
-t

Tensor(shape=[3, 3], dtype=Int64, value=
[[-1, -2, -3],
 [-4, -5, -6],
 [-7, -8, -9]])

In [42]:
theta = Parameter(np.zeros((1,)), requires_grad=True, name="theta")
phi = Parameter(np.zeros((1,)), requires_grad=True, name="phi")
a = ops.concat((ops.cos(theta) * ops.sin(phi), ops.sin(theta) * ops.sin(phi), ops.cos(phi)), axis=0)
a.param_info
theta.param_info

In [15]:
np.random.rand(1,) * np.pi * 2

array([3.80410183])